In [1]:
pip install paddlepaddle

     |████████████████████████████████| 204.9MB 70kB/s 


In [2]:
pip install paddle-ernie

  Created wheel for paddle-ernie: filename=paddle_ernie-0.1.0.dev1-cp37-none-any.whl size=20016 sha256=5dab3c3848c9e8f4c0a5cc96094c490a5928fa2ed21671aadfe42df5d688b0ca
  Stored in directory: /root/.cache/pip/wheels/ef/d8/52/d0fc0e19bce14963259ada61138e6c520805bd688e29652796
Successfully built paddle-ernie


In [3]:
import sys
!git clone https://github.com/PaddlePaddle/ERNIE.git
!git -C ./ERNIE branch origin/dygraph
sys.path.append('./ERNIE')
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import paddle as P
import paddle.fluid as F
import paddle.fluid.layers as L
import paddle.fluid.dygraph as D

from ernie.tokenizing_ernie import ErnieTokenizer
from ernie.modeling_ernie import ErnieModelForSequenceClassification

Cloning into 'ERNIE'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2034 (delta 1), reused 3 (delta 1), pack-reused 2027
Receiving objects: 100% (2034/2034), 88.86 MiB | 18.29 MiB/s, done.
Resolving deltas: 100% (1106/1106), done.


In [17]:
df_train_text = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/tweeteval/main/datasets/offensive/train_text.txt", delimiter="\t", names=["Tweet"])

In [18]:
df_train_label = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/tweeteval/main/datasets/offensive/train_labels.txt", delimiter="\t", names=["Label"])

In [19]:
df_train = df_train_text.join(df_train_label)
df_train.to_csv("train.csv", encoding='utf-8', index=False)
df_train

,Tweet,Label
0,@user Bono... who cares. Soon people will unde...,0
1,@user Eight years the republicans denied obama...,1
2,@user Get him some line help. He is gonna be j...,0
3,@user @user She is great. Hi Fiona!,0
4,@user She has become a parody unto herself? Sh...,1
...,...,...
11911,@user I wonder if they are sex traffic victims?,1
11912,@user Do we dare say he is... better than Nyjer?,0
11913,@user No idea who he is. Sorry,0
11914,#Professor Who Shot Self Over Trump Says Gun C...,0


In [20]:
df_test_text = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/tweeteval/main/datasets/offensive/test_text.txt", delimiter="\t", names=["Tweet"])

In [21]:
df_test_label = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/tweeteval/main/datasets/offensive/test_labels.txt", delimiter="\t", names=["Label"])

In [22]:
df_test = df_test_text.join(df_test_label)
df_test.to_csv("test.csv", encoding='utf-8', index=False)
df_test

,Tweet,Label
0,#ibelieveblaseyford is liar she is fat ugly li...,1
1,@user @user @user I got in a pretty deep debat...,0
2,"...if you want more shootings and more death, ...",0
3,Angels now have 6 runs. Five of them have come...,0
4,#Travel #Movies and Unix #Fortune combined Vi...,0
...,...,...
855,#CNN irrationally argues 4 legalising #abortio...,0
856,@user @user @user @user @user @user @user @use...,0
857,#Conservatives don’t care what you post..it’s ...,1
858,#antifa #Resist.. Trump is trying to bring wor...,0


In [10]:
#设置好所有的超参数，对于ERNIE任务学习率推荐取1e-5/2e-5/5e-5, 根据显存大小调节BATCH大小, 最大句子长度不超过512.
BATCH=32
MAX_SEQLEN=300
LR=5e-5
EPOCH=10

In [11]:
D.guard().__enter__() # 为了让Paddle进入动态图模式，需要添加这一行在最前面

ernie = ErnieModelForSequenceClassification.from_pretrained('ernie-2.0-en', num_labels=2)
optimizer = F.optimizer.Adam(LR, parameter_list=ernie.parameters())
tokenizer = ErnieTokenizer.from_pretrained('ernie-2.0-en')

downloading https://ernie-github.cdn.bcebos.com/model-ernie2.0-en.1.tar.gz: 873767KB [00:24, 35074.45KB/s]                            
/usr/local/lib/python3.7/dist-packages/ernie/modeling_ernie.py:265: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn('param:%s not set in pretrained model, skip' % k)
param:classifier.weight not set in pretrained model, skip
param:classifier.bias not set in pretrained model, skip


In [12]:
def make_data(path):
    data = []
    for i, l in enumerate(open(path)):
        if i == 0:
            continue
        l = l.strip().split(',')
        text, label = l[0], int(l[-1])
        text_id, _ = tokenizer.encode(text)
        text_id = text_id[:MAX_SEQLEN]
        text_id = np.pad(text_id, [0, MAX_SEQLEN-len(text_id)], mode='constant')
        label_id = np.array(label+1)
        data.append((text_id, label_id))
    return data

In [23]:
train_data = make_data('/content/train.csv')

In [24]:
train_data

Output hidden; open in https://colab.research.google.com to view.

In [25]:
test_data = make_data('/content/test.csv')

In [26]:
test_data

[(array([  101,  1001, 21307, 20806, 18697, 28522,  7952,  3877,  2003,
         16374,  2016,  2003,  6638,  9200, 21091,  2389,  1001,  4586,
         10258, 13808,  2016,  2853,  2014,  2841,  2000,  2131,  2070,
          5356,   999,   999,  2013, 17183,  2015,  1998,  4238,   999,
          2339,  2016,  3764,  2044,  1001,  2198,  5484,  2854,  7895,
          2078,  4168, 20624,  3070,  1029,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,

In [27]:
def get_batch_data(data, i):
    d = data[i*BATCH: (i + 1) * BATCH]
    feature, label = zip(*d)
    feature = np.stack(feature)  # 将BATCH行样本整合在一个numpy.array中
    label = np.stack(list(label))
    feature = D.to_variable(feature) # 使用to_variable将numpy.array转换为paddle tensor
    label = D.to_variable(label)
    return feature, label

In [ ]:
for i in range(EPOCH):
    np.random.shuffle(train_data) # 每个epoch都shuffle数据以获得最佳训练效果；
    #train
    for j in range(len(train_data) // BATCH):
        feature, label = get_batch_data(train_data, j)
        loss, _ = ernie(feature, labels=label) # ernie模型的返回值包含(loss, logits)；其中logits目前暂时不需要使用
        loss.backward()
        optimizer.minimize(loss)
        ernie.clear_gradients()
        if j % 10 == 0:
            print('train %d: loss %.5f' % (j, loss.numpy()))
        # evaluate
        if j % 100 == 0:
            all_pred, all_label = [], []
            with D.base._switch_tracer_mode_guard_(is_train=False): # 在这个with域内ernie不会进行梯度计算；
                ernie.eval() # 控制模型进入eval模式，这将会关闭所有的dropout；
                for j in range(len(test_data) // BATCH):
                    feature, label = get_batch_data(test_data, j)
                    loss, logits = ernie(feature, labels=label) 
                    all_pred.extend(L.argmax(logits, -1).numpy())
                    all_label.extend(label.numpy())
                ernie.train()
            f1 = f1_score(all_label, all_pred, average='macro')
            print('f1 %.5f' % f1)